Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [54]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [55]:
CITIES = pd.read_csv('cities/us.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Abilene,32.454514,-99.738147
1,Akron,41.080456,-81.521429
2,Albuquerque,35.105552,-106.647388
3,Alexandria,38.818343,-77.082026
4,Allen,33.107224,-96.674676


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [56]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## First Greedy Algorithm

In [57]:
#definisco una funzione per la prima versione, veloce e approssimativa (greedy):
def tsp_g():
    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    city = 0
    visited[city] = True
    tsp = [city]
    #cerco la  vicina
    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        logging.debug(
            f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city, closest]:.2f}km)"
        )
        visited[closest] = True
        city = closest
        tsp.append(city)
    #chiudo percorso
    tsp.append(tsp[0])
    logging.info(f"result: Found a path of {len(tsp) - 1} steps, total length {tsp_cost(tsp):.2f}km")
    return tsp



In [58]:
#seconda versione, più accurata, utilizzo nearest neighbor 
def tsp_nearest_neighbor_optimized():
    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    city = 0
    visited[city] = True
    tsp = [city]

    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        logging.debug(
            f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city, closest]:.2f}km)"
        )
        visited[closest] = True
        city = closest
        tsp.append(city)

    tsp.append(tsp[0])
    #come prima, adesso ottimizzo
    def opt(tour):
        best = tour
        improved = True
        while improved:
            improved = False
            for i in range(1, len(tour) - 2):
                for j in range(i + 1, len(tour) - 1):
                    if j - i == 1: continue
                    new_tour = best[:i] + best[i:j][::-1] + best[j:]
                    if tsp_cost(new_tour) < tsp_cost(best):
                        best = new_tour
                        improved = True
            tour = best
        return best

    optimized_tsp = opt(tsp)

    logging.info(f"result: Optimized path of {len(optimized_tsp) - 1} steps, total length {tsp_cost(optimized_tsp):.2f}km")
    return optimized_tsp


In [59]:
#avendo implementato entrambe le versioni a mo di funzione, stampiamo tutti i risultati assieme per confrontarli meglio. Per fare
#ciò vengono inizializzati le strutture all'interno di ogni funzione. 
fast_tsp = tsp_g()
print("fist version: ", fast_tsp)
print("n of steps: ", len(fast_tsp) - 1)
print("total cost: :", f"{tsp_cost(fast_tsp):.2f} km")

print("\n\n")

optimized_tsp = tsp_nearest_neighbor_optimized()
print("second version: ", optimized_tsp)
print("n of steps:", len(optimized_tsp) - 1)
print("total cost:", f"{tsp_cost(optimized_tsp):.2f} km")
#avranno stesso numero di step in quanto il numero di città da visitare è sempre quello


DEBUG:root:step: Abilene -> Wichita Falls (196.78km)
DEBUG:root:step: Wichita Falls -> Denton (149.68km)
DEBUG:root:step: Denton -> Lewisville (23.92km)
DEBUG:root:step: Lewisville -> Carrollton (10.02km)
DEBUG:root:step: Carrollton -> Plano (15.78km)
DEBUG:root:step: Plano -> Allen (9.27km)
DEBUG:root:step: Allen -> McKinney (10.45km)
DEBUG:root:step: McKinney -> Frisco (15.17km)
DEBUG:root:step: Frisco -> Richardson (22.38km)
DEBUG:root:step: Richardson -> Garland (10.04km)
DEBUG:root:step: Garland -> Mesquite (16.57km)
DEBUG:root:step: Mesquite -> Dallas (16.56km)
DEBUG:root:step: Dallas -> Irving (20.41km)
DEBUG:root:step: Irving -> Grand Prairie (19.83km)
DEBUG:root:step: Grand Prairie -> Fort Worth (32.28km)
DEBUG:root:step: Fort Worth -> Waco (136.07km)
DEBUG:root:step: Waco -> Killeen (74.58km)
DEBUG:root:step: Killeen -> Round Rock (61.72km)
DEBUG:root:step: Round Rock -> Austin (25.47km)
DEBUG:root:step: Austin -> San Antonio (118.67km)
DEBUG:root:step: San Antonio -> Corpus 

fist version:  [0, np.int64(321), np.int64(72), np.int64(154), np.int64(40), np.int64(225), np.int64(4), np.int64(168), np.int64(103), np.int64(240), np.int64(107), np.int64(173), np.int64(66), np.int64(133), np.int64(110), np.int64(100), np.int64(311), np.int64(141), np.int64(249), np.int64(18), np.int64(257), np.int64(64), np.int64(79), np.int64(167), np.int64(34), np.int64(148), np.int64(286), np.int64(219), np.int64(151), np.int64(126), np.int64(296), np.int64(57), np.int64(304), np.int64(273), np.int64(22), np.int64(143), np.int64(21), np.int64(174), np.int64(193), np.int64(181), np.int64(183), np.int64(60), np.int64(164), np.int64(15), np.int64(261), np.int64(14), np.int64(16), np.int64(270), np.int64(198), np.int64(45), np.int64(322), np.int64(94), np.int64(41), np.int64(234), np.int64(77), np.int64(114), np.int64(122), np.int64(323), np.int64(61), np.int64(46), np.int64(245), np.int64(3), np.int64(12), np.int64(313), np.int64(59), np.int64(20), np.int64(222), np.int64(5), np.in

DEBUG:root:step: Henderson -> Sunrise Manor (18.48km)
DEBUG:root:step: Sunrise Manor -> North Las Vegas (12.15km)
DEBUG:root:step: North Las Vegas -> Las Vegas (16.86km)
DEBUG:root:step: Las Vegas -> Surprise (382.61km)
DEBUG:root:step: Surprise -> Phoenix (35.56km)
DEBUG:root:step: Phoenix -> Tempe (25.02km)
DEBUG:root:step: Tempe -> Chandler (13.72km)
DEBUG:root:step: Chandler -> Gilbert (10.93km)
DEBUG:root:step: Gilbert -> Mesa (10.43km)
DEBUG:root:step: Mesa -> Scottsdale (31.20km)
DEBUG:root:step: Scottsdale -> Tucson (190.13km)
DEBUG:root:step: Tucson -> Santa Maria (940.50km)
DEBUG:root:step: Santa Maria -> Santa Rosa (439.17km)
DEBUG:root:step: Santa Rosa -> Reno (278.35km)
DEBUG:root:step: Reno -> Sparks (12.12km)
DEBUG:root:step: Sparks -> Billings (1144.36km)
DEBUG:root:step: Billings -> Fargo (909.58km)
DEBUG:root:step: Fargo -> Des Moines (641.39km)
DEBUG:root:step: Des Moines -> Cedar Rapids (167.02km)
DEBUG:root:step: Cedar Rapids -> Davenport (100.38km)
DEBUG:root:step

second version:  [0, np.int64(321), np.int64(72), np.int64(154), np.int64(40), np.int64(103), np.int64(168), np.int64(4), np.int64(225), np.int64(240), np.int64(107), np.int64(173), np.int64(66), np.int64(133), np.int64(110), np.int64(100), np.int64(311), np.int64(141), np.int64(249), np.int64(18), np.int64(257), np.int64(148), np.int64(167), np.int64(79), np.int64(34), np.int64(64), np.int64(286), np.int64(219), np.int64(151), np.int64(126), np.int64(296), np.int64(57), np.int64(304), np.int64(273), np.int64(22), np.int64(143), np.int64(21), np.int64(174), np.int64(193), np.int64(134), np.int64(157), np.int64(169), np.int64(26), np.int64(15), np.int64(261), np.int64(14), np.int64(16), np.int64(164), np.int64(60), np.int64(183), np.int64(181), np.int64(292), np.int64(105), np.int64(281), np.int64(54), np.int64(253), np.int64(293), np.int64(244), np.int64(30), np.int64(144), np.int64(38), np.int64(153), np.int64(69), np.int64(220), np.int64(180), np.int64(176), np.int64(120), np.int64(1